## Tornado的Etag

### 如何禁用Etag

重载compute_etag方法，并返回None即可.

建议:对于所有动态页面，不用计算etag的方法，这样可以让客户端永远获取最新的内容。

如果真的要客户端缓存页面，可以用Cache-Control，设置有效期，这样比较合理。


其实不用管ETag,tornado中计算ETag的值是将上次的ETag的值与最新数据的MD5值后比较,(Tornado计算Etag是用的md5算法)如果一致就返回304,不一致就返回最新的数据.这实际上一种优化.

## Tornado用户身份校验

tornado提供了一个装饰器@web.authenticated,用来进行用户身份校验,这个装饰器分别装饰get和post方法.

### 如何设置用户身份校验

用户的身份校验一般是根据cookies来进行校验的,一般在用户登陆的时候,在服务器端产生一个cookies,同时保存在客户端浏览器上,在用户发出的每次请求都会自动在请求header中包含这个cookies.

### 关于secret_cookie

一般在用户成功登录后设置相应的cookies,在设置secret_cookie需要指定一个字段,比如uid

In [ ]:
import logging
from app import utils
from app.models import usermodel
from app.ctrls.basic import BaseHandler


class LoginHandler(BaseHandler):

    def get(self):
        self.post()

    def post(self):
        username = self.get_argument('username', None)
        password = self.get_argument('password', None)
        # 检测用户名合法性
        if not utils.check_username_verify(username):
            logging.error('username or password is error, please try again.')
            self.write({'status': '406', 'msg': 'login failed'})
        # 检测用户密码合法性
        elif not utils.check_password_verify(password):
            logging.error('username or password is error, please try again.')
            self.write({'status': '406', 'msg': 'login failed'})
        # 检验用户名和密码是否正确
        elif usermodel.check_user_verify(username, password):
            # 根据传进的用户名和密码在数据库中查询用户名和密码是否正确
            uid = usermodel.get_uid_by_name(username)
            # 设定secret_cookie
            if uid is not None:
                self.set_secure_cookie('uid', str(uid), expires_days=1)
                self.write({'status': '200', 'msg': 'login success'})
            else:
                logging.error('Set cookie failed')
                self.write({'status': '406', 'msg': 'login failed'})
        else:
            logging.error('username or password is error, please try again.')
            self.write({'status': '406', 'msg': 'login failed'})
        self.finish()

tornado使用set_secret_cookie()设置用户的secret_cookie,这个secret_cookie是加密的,在tornado的配置项cookie_secret设置你的密匙,这个密匙无论如何都是不能泄露,否则没有任何安全性可言.

In [ ]:
settings = {
    'xsrf_cookies': True,
    'cookie_secret': 'do not try to guess!', # 在这里设置你的密匙
    'login_url': '/api/login',
    'static_path': os.path.join(os.path.dirname(os.path.dirname(__file__)), 'static')
}

设置好密匙然后登陆成功后,在请求头中你就会发现被加了密的cookie,只要不知道你的密匙,别人是无法知道你真正的cookie中字段值,如下所示:

Cookie: uid="2|1:0|10:1522378973|3:uid|4:Mw==|174f66a78882002413f16c01c0c19ef6395108dbd2e0ff38462f487de1385675"

### tornado是如何在每个请求时校验身份

**Tips:** 一般我们会自定义一个BaseHandler,然后其他的handler都从这个BaseHandler继承,这样把所有公共的操作和属性都放到父类中,这样从父类继承的子类就会都带有父类的操作和属性,从而减少重复性工作.比如定制header等

In [ ]:
from tornado import web

class BaseHandler(web.RequestHandler):

    def initialize(self):
        pass

    def set_default_headers(self):
        pass

    def get_current_user(self):
        """"重载父类RequestHandler中的get_current_user,函数名不能改变, 做校验用的"""
        uid = self.get_secure_cookie('uid')
        return uid.decode()

tornado校验用户身份是通过重写`get_current_user()`这个方法来实现的,这个`get_current_user()`是固定死的,只能重写,当你重写这个函数,tornado框架会在发生请求时自动先调用这个函数来检验用户的身份.

我们之前在登陆时调用了`set_secure_cookie('uid', str(uid), expires_days=1)`,设置了`uid`字段和过期时间并保存在服务端,当发生请求后,服务端就会解析从客户端浏览器发来的请求,即调用`get_current_user()`这个方法.当cookie不一致(可能被篡改)或者过期,则会发生页面跳转.
* 当客户端cooike值和服务端的cookie值不一致时,则判断不是当前用户,tornado就会跳转到一个指定页面,(这个路由可以在配置中指定`login_url`),
* 当cookie过期时,调用`get_current_user()`则会返回一个`None`,同时页面跳转到一个指定页面(这个路由可以在配置中指定`login_url`)

@web.authenticated装饰器校验用户身份,其实就是调用的`get_current_user()`函数.

### 获取cookies的值

1. get_cookie: 获取的是加密后的cookie值
2. get_secure_cookie: 获取是的明文cookie,但是值是bytes类型,需要decode
3. get_secure_cookie('uid').decode(): 获取cookie字符串明文